In [2]:
# 모듈을 불러온다. 
import pandas as pd
from pandas import Series, DataFrame
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import konlpy
import nltk
from konlpy.tag import Kkma;kkma = Kkma()
from konlpy.corpus import kobill

In [3]:
#--------------NewsCrawling------------------#
# 특정 기사의 본문과 제목을 가져오는 함수. 
def NewsCraw(url):
    html = urlopen(url) # url에서 html을 가져온다. 
    soup = BeautifulSoup(html,"html.parser", from_encoding = "utf-8") # bs4에서 파싱할 수 있는 구조로 바꾼다.
    content = soup.find(id = 'articleBodyContents') # id로 본문 내용을 가져온다. 
    hangul = re.compile("[^ ㄱ-ㅎ|가-힣.""]+") # 한글,.,띄어쓰기가 아닌 문자(영어, 특수문자, 숫자 등등)를 나타내는 정규표현식을 만든다.
    content = str(content) # html 객체?를 문자열로 바꾼다.
    content = hangul.sub("",content) # 정규표현식에 해당하는 패턴을 지운다. 
    content = re.sub("본문 내용  플레이어   플레이어    오류를 우회하기 위한 함수 추가","",content) # 불필요한 한글 패턴 제거
    content = re.sub('본문 내용  플레이어   동영상 뉴스       영상 플레이어   플레이어    오류를 우회하기 위한 함수 추가','',content)
    content = re.sub("본문 내용  플레이어   동영상 뉴스 .      영상 플레이어   플레이어    오류를 우회하기 위한 함수 추가",'',content)
    title = soup.find("h3",{"id":"articleTitle"}).find(text=True) # 제목을 가져온다. 
    return [title ,content] # 제목과 내용을 반환한다. 

# 기사목록에서 개별기사를 가져오는 함수.
def articles(url):
    articles = []
    for i in range(1,6):
        page_url = url+str(i) # i를 문자열로 변경.
        page_html = urlopen(page_url) 
        page_soup = BeautifulSoup(page_html,"html.parser",from_encoding = "utf-8")
        articles.extend(page_soup.findAll("dt"))
        url_list = []
        for link in articles:
            url_list.append(link.find("a")["href"])
            url_list = list(set(url_list))
    return url_list

# 기사에서 제목과 내용을 가져오는 함수.
def ticon(url_list):
    df = DataFrame(columns=("기사제목","본문내용"))
    for i, url in enumerate(url_list):
        df.loc[i] = NewsCraw(url)
    return df

# 뉴스 주제별로 기사목록을 가져온다. 
pol_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20181010&page=" 
cul_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2=245&sid1=103&mid=shm&date=20181010&page="
sci_url = "https://news.naver.com/main/list.nhn?mode=LS2D&sid2=228&sid1=105&mid=shm&date=20181010&page="

# 토픽에 따라 기사들을 정리. 
pol_articles = articles(pol_url)
cul_articles = articles(cul_url)
sci_articles = articles(sci_url)

pol_articles = ticon(pol_articles)
cul_articles = ticon(cul_articles)
sci_articles = ticon(sci_articles)

In [4]:
#--------------LDA------------------#
# Tokenize
pol_doc = []
for i in range(0,len(pol_articles)):
    pol_doc.append(kkma.morphs(pol_articles.iloc[i,1]))

cul_doc = []
for i in range(0,len(cul_articles)):
    cul_doc.append(kkma.morphs(cul_articles.iloc[i,1]))

sci_doc = []
for i in range(0,len(sci_articles)):
    sci_doc.append(kkma.morphs(sci_articles.iloc[i,1]))

len(pol_doc) #100
len(cul_doc) #100
len(sci_doc) #81

# LDA
import random
from collections import Counter

documents = []
documents.extend(pol_doc)
documents.extend(cul_doc)
documents.extend(sci_doc)
len(documents) # 1~100까지는 정치 101~200까지는 문화 200~281은 과학

281

In [7]:
#조건부 확률 분포 정의를 위한 준비

#topic의 개수
K = 3

#1. 각 토픽이 각 문서에 할당되는 횟수
#counter로 구성된 list
#각각의 counter는 각 문서를 의미함
document_topic_counts = [Counter() for _ in documents]

#2. 각 단어가 각 토픽에 할당되는 횟수
# 각각의 counter는 각 토픽을 의미함
topic_word_counts = [Counter() for _ in range(K)] 

#3. 각 토픽에 할당되는 총 단어 수
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)] 

#4. 각 문서에 포함되는 총 단어의 수
# 각각의 숫자는 각 문서를 의미함
document_lengths = [len(d) for d in documents]

#5. 단어 종류의 수
distinct_words = set(word for document in documents for word in document) 
W = len(distinct_words)

#6. 총 문서의 수 
D = len(documents)

In [ ]:
#### 새로운 topic 계산하기

def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

#랜덤으로 생성된 weight로부터 인덱스를 생성함
def sample_from(weights):
     total = sum(weights)
     rnd = total * random.random()       # uniform between 0 and total
     for i, w in enumerate(weights):
         rnd -= w                        # return the smallest i such that
         if rnd <= 0: return i           # sum(weights[:(i+1)]) >= rnd

random.seed(0)

#topic의 개수
K = 3

# 각 단어를 임의의 토픽에 배정
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

# 랜덤 초기화한 상태에서 AB를 구하는 데 필요한 숫자 계산하기
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

# 조건부 확률 분포를 이용하여 (토픽-단어), (문서-토픽)에 대한 깁스 샘플링 실행하기
for iter in range(1000): 
    print(iter)
    for d in range(D): 
        for i, (word, topic) in enumerate(zip(documents[d], 
                                              document_topics[d])): 
 
 
           # remove this word / topic from the counts
           # so that it doesn't influence the weights 
            document_topic_counts[d][topic] -= 1 
            topic_word_counts[topic][word] -= 1 
            topic_counts[topic] -= 1 
            document_lengths[d] -= 1 
 
           # choose a new topic based on the weights 
            new_topic = choose_new_topic(d, word) 
            document_topics[d][i] = new_topic 

 
           # and now add it back to the counts 
            document_topic_counts[d][new_topic] += 1 
            topic_word_counts[new_topic][word] += 1 
            topic_counts[new_topic] += 1 
            document_lengths[d] += 1

#각 토픽에 가장 영향력이 높은 (weight)값이 큰 단어 탐색
for k, word_counts in enumerate(topic_word_counts): 
         for word, count in word_counts.most_common(): 
             if count > 0: print (k, word, count) 

In [12]:
topic_names = ["문화", "정치", "과학"]

df = DataFrame(columns=("토픽","횟수","문서번호"))
num = -1 
for document, topic_counts in zip(documents, document_topic_counts): 
        num += 1
        for topic, count in topic_counts.most_common():
            temp = pd.DataFrame({"토픽":topic_names[topic],"횟수":count,"문서번호":num}, index = [0])
            df = df.append(temp)

df = df.reset_index(drop = True)

pred_topics = []
for i,_ in enumerate(df.T):
    if i%3 == 0:
        df2 = df[i:i+3]
        topic = df2["토픽"][df2["횟수"].astype('float64').idxmax()] # 실수로 바꺼야 idmax가 먹힌다...
        pred_topics.append(topic)

len(pred_topics) #281

print(Counter(pred_topics[0:100])) # 실제 정치 기사
print(Counter(pred_topics[100:200]))# 실제 문화 기사
print(Counter(pred_topics[200:280])) # 실제 과학 기사

Counter({'정치': 87, '과학': 11, '문화': 2})
Counter({'문화': 72, '과학': 18, '정치': 10})
Counter({'과학': 70, '정치': 9, '문화': 1})


Counter({'정치': 87, '과학': 11, '문화': 2}) <br>
Counter({'문화': 72, '과학': 18, '정치': 10}) <br>
Counter({'과학': 70, '정치': 9, '문화': 1}) <br>

LDA로 문서를 잘 구분한다. 특히 과학기사를 잘 구분한다. 아무래도 정치/문화 보다 전문적인 용어가 많고 기사가 서로 겹치지 않아서 그런 듯 하다. <br> 
하지만 몇몇 기사는 잘 구분하지 못하는 데, 세션별로 기사가 겹치는 경우가 있기 때문으로 생각된다. 

In [ ]:
#--------------Word2Vec------------------#
# Word2Vec 모델 만들기
from gensim.models.word2vec import Word2Vec
model = Word2Vec(documents,size=500, window=10, min_count=5, sg=1) # 여러번 해보니깐 모수 조절이 중요한듯? 어떤 기준으로 정해야할까..
model.init_sims(replace=True)

print(model.similarity('뮤지컬', '영화'))
print(model.similarity('여당', '대통령'))
print(model.similarity('연구', '과학'))
print(model.similarity('남북', '뮤지컬'))
print(model.similarity('민주당', '축제'))

print(model.most_similar("북한")) # 파라미터들을 코드 대로 size=100, window=3, min_count=2, sg=1 했을 때는 최빈값이 부동산..이었음.. 파라미터를 조절하니깐 합리적으로 나온다! 파라미터 조절을 어떻게 해야할까..

print(model.most_similar(positive=['북한', '비핵화'], negative='야당', topn=1))